In [ ]:
import os
import imageio.v2 as imageio
import torch
from PIL import Image

img_arr = imageio.imread(os.path.join(os.path.pardir, "_00_data", "a_image-dog", "bobby.jpg")) #파일열기

print(type(img_arr))  # 이미지 배열의 데이터 타입 출력
print(img_arr.shape)  # 이미지의 형태 출력
print(img_arr.dtype)  # 이미지의 데이터 타입 출력

img = torch.from_numpy(img_arr) # numpy 배열을 PyTorch 텐서로 변환
out = img.permute(2, 0, 1)  # 이미지의 차원을 (높이, 너비, 채널)에서 (채널, 높이, 너비)로 변환
print(out.shape)  

print("#" * 50, 1)

data_dir = os.path.join(os.path.pardir, "_00_data", "b_image-cats")

# 해당 디렉토리 내에 있는 .png 확장자의 파일명을 가져옴
filenames = [
  name for name in os.listdir(data_dir) if os.path.splitext(name)[-1] == '.png'
]
print(filenames)


# 각 파일을 열어서 이미지 출력 및 정보 출력
for i, filename in enumerate(filenames):
  image = Image.open(os.path.join(data_dir, filename))
  image.show()
  img_arr = imageio.imread(os.path.join(data_dir, filename))
  print(img_arr.shape)
  print(img_arr.dtype)

batch_size = 3  # 배치 사이즈 설정 
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8) # (배치, 채널, 높이, 너비)의 빈 텐서 생성


for i, filename in enumerate(filenames):
  img_arr = imageio.imread(os.path.join(data_dir, filename))
  img_t = torch.from_numpy(img_arr)
  img_t = img_t.permute(2, 0, 1)
  batch[i] = img_t  # 변환된 텐서를 배치에 추가

print(batch.shape)  # 배치 텐서의 크기 출력

print("#" * 50, 2)

batch = batch.float()
batch /= 255.0  #이미지값을 0~1로 정규화
print(batch.dtype)
print(batch.shape)

n_channels = batch.shape[1]

for c in range(n_channels):
  mean = torch.mean(batch[:, c])
  std = torch.std(batch[:, c])
  print(mean, std)
  batch[:, c] = (batch[:, c] - mean) / std

# A 고찰내용
#### 이미지 데이터 확인 및 출력
1. 이미지 파일을 텐서로 관리할 수 있다.
2. 텐서 차원의 순서를 변경할 수 있다.
3. 정규화를 할 수 있다.

In [ ]:
import os

import imageio.v2 as imageio

# DICOM 볼륨 데이터가 있는 디렉토리 경로 설정
dir_path = os.path.join(os.path.pardir, "_00_data", "c_volumetric-dicom", "2-LUNG_3.0_B70f-04083")  # 파일 경로 코드 수정
# DICOM 볼륨 데이터를 읽어 numpy 배열로 변환
vol_array = imageio.volread(dir_path, format='DICOM')
print(type(vol_array))   # >>> <class 'imageio.core.util.Array'>:  Numpy NDArray
# 이는 99개의 슬라이스(slice)로 구성된 512x512 크기의 3D 이미지임을 의미함
print(vol_array.shape)   # >>> (99, 512, 512)
# vol_array의 데이터 타입 출력: DICOM 이미지 파일에서 추출된 int16 데이터 타입
print(vol_array.dtype)   # >>> int16
print(vol_array[0])

print("#" * 50, 1)

import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10, 10))
for id in range(0, 99):
  fig.add_subplot(10, 10, id + 1)
  plt.imshow(vol_array[id])
plt.show()  # 전체 슬라이스 이미지들을 한 번에 표시

import torch
# numpy 배열을 PyTorch 텐서로 변환하고, float 타입으로 변환
vol = torch.from_numpy(vol_array).float()
# 첫 번째 차원에 채널 차원을 추가 (예: [C, D, H, W] 형태로 만듦)
vol = torch.unsqueeze(vol, 0)  # channel
# 두 번째 차원에 데이터 차원을 추가하여 [B, C, D, H, W] 형태로 만듦
vol = torch.unsqueeze(vol, 0)  # data size

print(vol.shape)  # >>> torch.Size([1, 1, 99, 512, 512])
  
print("#" * 50, 2)

mean = torch.mean(vol, dim=(3, 4), keepdim=True)  # mean over all of dim=(3, 4)
print(mean.shape)
std = torch.std(vol, dim=(3, 4), keepdim=True)    # std over all of dim=(3, 4)
print(std.shape)
vol = (vol - mean) / std
print(vol.shape)

print(vol[0, 0, 0])  

# B 고찰내용
1. DICOM 데이터의 읽기 및 numpy 배열 변환을 할 수 있다.
2. 데이터 형태 및 타입 확인할 수 있다.
3. 이미지 시각화를 할 수 있다.
4. numpy를 텐서로 변환할 수 있다.
5. 정규화를 할 수 있다.
6. 3D 볼륨 데이터에서 특정 슬라이스에 접근할 수 있다.

In [ ]:
import csv
import os
import numpy as np

wine_path = os.path.join(os.path.pardir, "_00_data", "d_tabular-wine", "winequality-white.csv")
# 데이터를 NumPy 배열로 로드 (헤더를 건너뛰고, ';'로 구분된 값)
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";", skiprows=1)
print(wineq_numpy.dtype)
print(wineq_numpy.shape)
print(wineq_numpy)
print()
# CSV 파일에서 컬럼 목록 읽기
col_list = next(csv.reader(open(wine_path), delimiter=';'))
print(col_list)
print()

print("#" * 50, 1)

import torch
# NumPy 배열을 PyTorch 텐서로 변환
wineq = torch.from_numpy(wineq_numpy)
print(wineq.dtype)
print(wineq.shape)
print()

# 특성과 타겟을 분리
data = wineq[:, :-1]  # Selects all rows and all columns except the last
print(data.dtype)
print(data.shape)
print(data)
print()

target = wineq[:, -1]  # Selects all rows and the last column
print(target.dtype)
print(target.shape)
print(target)
print()

target = target.long()  # treat labels as an integer
print(target.dtype)
print(target.shape)
print(target)
print()

print("#" * 50, 2)

# 원-핫 인코딩을 위한 단위 행렬 생성
eye_matrix = torch.eye(10)
# We use the 'target' tensor as indices to extract the corresponding rows from the identity matrix
# It can generate the one-hot vectors for each element in the 'target' tensor
onehot_target = eye_matrix[target]

print(onehot_target.shape)  # >>> torch.Size([4898, 10])
print(onehot_target[0])
print(onehot_target[1])
print(onehot_target[-2])
print(onehot_target)

print("#" * 50, 3)
# 데이터 정규화
data_mean = torch.mean(data, dim=0)
data_var = torch.var(data, dim=0)
data = (data - data_mean) / torch.sqrt(data_var)
print(data)

print("#" * 50, 4)

from sklearn.model_selection import train_test_split
# 데이터 분할: 훈련 세트와 검증 세트
X_train, X_test, y_train, y_test = train_test_split(data, onehot_target, test_size=0.2)

print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)


def get_wine_data():  #함수화
  # 데이터 로드 및 전처리 함수 정의
  wine_path = os.path.join(os.path.pardir, os.path.pardir, "_00_data", "d_tabular-wine", "winequality-white.csv")
  wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";", skiprows=1)
  
  # NumPy 배열을 PyTorch 텐서로 변환
  wineq = torch.from_numpy(wineq_numpy)
  # 특성과 타겟을 분리
  data = wineq[:, :-1]  # Selects all rows and all columns except the last
  target = wineq[:, -1].long()  # treat labels as an integer
  # 원-핫 인코딩
  eye_matrix = torch.eye(10)
  onehot_target = eye_matrix[target]
  # 데이터 정규화
  data_mean = torch.mean(data, dim=0)
  data_var = torch.var(data, dim=0)
  data = (data - data_mean) / torch.sqrt(data_var)

  X_train, X_valid, y_train, y_valid = train_test_split(data, onehot_target, test_size=0.2)

  return X_train, X_valid, y_train, y_valid

# C 고찰내용
1. csv 파일을 읽고 로드할 수 있다.
2. 타겟을 정수형으로 변환하고 원 핫 인코딩 후 정규화할 수 있다.
* 원 핫 인코딩은 보통 범주형 데이터를 수치형으로 바꿀 때 사용한다.
3. 데이터분할이 가능하다.
4. 함수화하여 사용 가능하다.

In [ ]:
# https://medium.com/analytics-vidhya/implement-linear-regression-on-boston-housing-dataset-by-pytorch-c5d29546f938
# https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset
import torch
from sklearn.datasets import fetch_california_housing
# California Housing 데이터셋을 로드합니다.
housing = fetch_california_housing()
print(housing.keys()) # 데이터셋의 키를 출력하여 어떤 정보가 포함되어 있는지 확인합니다.

# 데이터와 관련된 정보 출력
print(type(housing.data))  # 데이터 타입 확인 (일반적으로 <class 'numpy.ndarray'>)
print(housing.data.dtype)  # 데이터의 타입 확인 (예: float64)
print(housing.data.shape)  # 데이터 배열의 형태 확인 (예: (20640, 8))
print(housing.feature_names)  # 데이터의 특성 이름 출력

print(housing.target.shape)  # 타겟 배열의 형태 확인 (예: (20640,))
print(housing.target_names)  # 타겟의 이름 출력 (California Housing 데이터는 타겟 이름이 없음)

print("#" * 50, 1)

import numpy as np

print(housing.data.min(), housing.data.max())
#정규화
data_mean = np.mean(housing.data, axis=0)
data_var = np.var(housing.data, axis=0)
data = (housing.data - data_mean) / np.sqrt(data_var)
target = housing.target


print(data.min(), data.max()) # 정규화된 데이터 최소값과 최대값을 출력하여 정규화 확인.

print("#" * 50, 2)

from sklearn.model_selection import train_test_split
# 데이터를 훈련 세트와 테스트 세트로 나눕니다.
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)

X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

# 훈련 데이터와 테스트 데이터의 형태를 출력하여 확인합니다.
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

# D 고찰내용
1. fetch_california_housing() 함수를 통해 California Housing 데이터셋을 로드할 수 있다.
2. 정규화의 필요성
* 데이터의 범위가 서로 다를 경우, 모델학습의 성능을 향상시키기 위해 정규화가 필요하다.
* 방법 : 평균과 분산계산 ->  정규화 적용 -> 정규화 확인
3. 훈련 및 테스트 데이터로 분할 한다.
* 모델의 일반화 성능 평가와 과적합 방지를 위해 분할한다. 
4. 훈련데이터와 테스트 데이터의 형태를 출력하여 검사할 필요가 있다.

In [ ]:
import os
import numpy as np
import torch

torch.set_printoptions(edgeitems=2, threshold=50, linewidth=75) # PyTorch의 출력 옵션 설정: 배열의 끝부분을 2개 항목만 표시하고, 전체 출력 개수 제한

bikes_path = os.path.join(os.path.pardir, "_00_data", "e_time-series-bike-sharing-dataset", "hour-fixed.csv")

bikes_numpy = np.loadtxt(   # CSV 파일에서 데이터를 NumPy 배열로 로드
  fname=bikes_path, dtype=np.float32, delimiter=",", skiprows=1,
  converters={
    1: lambda x: float(x[8:10])  # 2011-01-07 --> 07 --> 7.0
  }
)
bikes = torch.from_numpy(bikes_numpy)
print(bikes.shape)

# 데이터를 하루 단위로 재구성 (730일, 24시간, 17개의 특성)
daily_bikes = bikes.view(-1, 24, bikes.shape[1])
print(daily_bikes.shape)  # >>> torch.Size([730, 24, 17])
# 특성과 타겟을 분리
daily_bikes_data = daily_bikes[:, :, :-1]
daily_bikes_target = daily_bikes[:, :, -1].unsqueeze(dim=-1)

print(daily_bikes_data.shape)
print(daily_bikes_target.shape)

print("#" * 50, 1)
# 첫 번째 날의 데이터 추출
first_day_data = daily_bikes_data[0]
print(first_day_data.shape)

# 날씨 상황 열을 정수형으로 출력
print(first_day_data[:, 9].long())
# 날씨 상황을 원-핫 인코딩하기 위한 단위 행렬 생성
eye_matrix = torch.eye(4)
print(eye_matrix)

# 날씨 상황 열을 원-핫 인코딩
weather_onehot = eye_matrix[first_day_data[:, 9].long() - 1]
print(weather_onehot.shape)
print(weather_onehot)

# 원-핫 인코딩된 날씨 데이터를 기존의 데이터와 연결
first_day_data_torch = torch.cat(tensors=(first_day_data, weather_onehot), dim=1)
print(first_day_data_torch.shape)
print(first_day_data_torch)

print("#" * 50, 2)

# 모든 날에 대해 날씨 상황을 원-핫 인코딩하고, 기존 데이터와 연결
day_data_torch_list = []

for daily_idx in range(daily_bikes_data.shape[0]):  # range(730)
  day = daily_bikes_data[daily_idx]  # day.shape: [24, 16]
  weather_onehot = eye_matrix[day[:, 9].long() - 1]
  day_data_torch = torch.cat(tensors=(day, weather_onehot), dim=1)  # day_data_torch.shape: [24, 20]
  day_data_torch_list.append(day_data_torch)

print(len(day_data_torch_list))
daily_bikes_data = torch.stack(day_data_torch_list, dim=0)
print(daily_bikes_data.shape)

print("#" * 50, 3)
# 데이터에서 'instant'와 'wheathersit' 열을 제거하고 나머지 열을 사용
print(daily_bikes_data[:, :, :9].shape, daily_bikes_data[:, :, 10:].shape)
daily_bikes_data = torch.cat(
  [daily_bikes_data[:, :, 1:9], daily_bikes_data[:, :, 10:]], dim=2
) # Drop 'instant' and 'whethersit' columns
print(daily_bikes_data.shape)

# 온도 열을 정규화 (평균이 0, 표준편차가 1이 되도록 변환)
temperatures = daily_bikes_data[:, :, 8]
daily_bikes_data[:, :, 8] = (daily_bikes_data[:, :, 8] - torch.mean(temperatures)) / torch.std(temperatures)


# E 고찰 내용
1. 데이터 재구성
* bikes.view(-1, 24, bikes.shape[1])로 데이터셋을 하루단위로 재구성할 수 있다.
* 특성과 타겟을 분리하고 나머지 열을 특성으로 사용가능하다.
2. 원 핫 인코딩
* torch.eye(4)를 통해 단위 행렬을 생성할 수 있다.
* eye_matrix[first_day_data[:, 9].long() - 1]를 통해 날씨 상황 열을 원-핫 인코딩하여 범주형 변수를 수치형 벡터로 변환할 수 있다.
* torch.cat()로 기존 데이터와 원 핫 인코딩된 데이터를 합칠 수 있다.
3. 데이터 정리 및 처리
* 모든 날의 데이터를 리스트로 저장하고, torch.stack를 사용하여 텐서로 변환할 수 있다.
* torch.cat를 사용하여 불필요한 열을 제거할 수 있다.

In [ ]:
import os
import numpy as np
import torch
from pathlib import Path

BASE_PATH = str(Path("Deep_HW").resolve().parent.parent) # 파일경로 수정
import sys
# BASE_PATH를 시스템 경로에 추가합니다.
# 이렇게 하면 BASE_PATH에 있는 모듈들을 임포트할 수 있다.
sys.path.append(BASE_PATH)

torch.set_printoptions(edgeitems=2, threshold=50, linewidth=75)

bikes_path = os.path.join(BASE_PATH, "_00_data", "e_time-series-bike-sharing-dataset", "hour-fixed.csv")

bikes_numpy = np.loadtxt(
  fname=bikes_path, dtype=np.float32, delimiter=",", skiprows=1,
  converters={
    1: lambda x: float(x[8:10])  # 2011-01-07 --> 07 --> 7
  }
)

bikes_data = torch.from_numpy(bikes_numpy).to(torch.float)
print(bikes_data.shape)    # 텐서의 형태 출력; torch.Size([17520, 17])
bikes_target = bikes_data[:, -1].unsqueeze(dim=-1)  # 'cnt' 열을 타겟으로 설정
bikes_data = bikes_data[:, :-1]   # 마지막 열 'cnt'를 제외한 나머지 데이터 사용; torch.Size([17520, 16])

# 날씨 정보를 원-핫 인코딩하기 위한 단위 행렬 생성
eye_matrix = torch.eye(4)

# 데이터를 처리하여 원-핫 인코딩을 적용
data_torch_list = []
for idx in range(bikes_data.shape[0]):  # range(730)
  hour_data = bikes_data[idx]  # 시간 단위 데이터 추출
  weather_onehot = eye_matrix[hour_data[9].long() - 1]
  concat_data_torch = torch.cat(tensors=(hour_data, weather_onehot), dim=-1)
  # concat_data_torch.shape: [20]
  data_torch_list.append(concat_data_torch)

# 처리된 데이터를 하나의 텐서로 변환하고, 필요 없는 열을 제거
bikes_data = torch.stack(data_torch_list, dim=0)
bikes_data = torch.cat([bikes_data[:, 1:9], bikes_data[:, 10:]], dim=-1)
# Drop 'instant' and 'whethersit' columns

print(bikes_data.shape)
print(bikes_data[0])

#################################################################################################

# 시퀀스 크기, 검증 크기, 테스트 크기 설정
sequence_size = 24
validation_size = 96
test_size = 24
y_normalizer = 100

# 전체 데이터 크기와 훈련, 검증, 테스트 크기 계산
data_size = len(bikes_data) - sequence_size + 1
print("data_size: {0}".format(data_size))
train_size = data_size - (validation_size + test_size)
print("train_size: {0}, validation_size: {1}, test_size: {2}".format(train_size, validation_size, test_size))

print("#" * 50, 1)

#################################################################################################
# 훈련 데이터를 생성
row_cursor = 0

X_train_list = []
y_train_regression_list = []
for idx in range(0, train_size):
  sequence_data = bikes_data[idx: idx + sequence_size]  # 시퀀스 데이터를 추출
  sequence_target = bikes_target[idx + sequence_size - 1] # 시퀀스의 마지막 값을 타겟으로 설정
  X_train_list.append(sequence_data)
  y_train_regression_list.append(sequence_target)
  row_cursor += 1

X_train = torch.stack(X_train_list, dim=0).to(torch.float)  # 훈련 데이터 텐서로 변환  
print(X_train.shape)
y_train_regression = torch.tensor(y_train_regression_list, dtype=torch.float32) / y_normalizer  # 타겟 데이터 정규화

# 훈련 데이터 정규화
m = X_train.mean(dim=0, keepdim=True)
s = X_train.std(dim=0, keepdim=True)
X_train = (X_train - m) / s

print(X_train.shape, y_train_regression.shape)
# >>> torch.Size([17376, 24, 19]) torch.Size([17376])

print("#" * 50, 2)
#################################################################################################
# 검증 데이터를 생성
X_validation_list = []
y_validation_regression_list = []
for idx in range(row_cursor, row_cursor + validation_size):
  sequence_data = bikes_data[idx: idx + sequence_size]  # 시퀀스 데이터를 추출
  sequence_target = bikes_target[idx + sequence_size - 1]
  X_validation_list.append(sequence_data)
  y_validation_regression_list.append(sequence_target)
  row_cursor += 1

X_validation = torch.stack(X_validation_list, dim=0).to(torch.float)
y_validation_regression = torch.tensor(y_validation_regression_list, dtype=torch.float32) / y_normalizer

X_validation = (X_validation - m) / s

print(X_validation.shape, y_validation_regression.shape)
# >>> torch.Size([96, 24, 19]) torch.Size([96])

print("#" * 50, 3)
#################################################################################################
# 테스트 데이터를 생성
X_test_list = []
y_test_regression_list = []
for idx in range(row_cursor, row_cursor + test_size):
  sequence_data = bikes_data[idx: idx + sequence_size]  # 시퀀스 데이터를 추출
  sequence_target = bikes_target[idx + sequence_size - 1]
  X_test_list.append(sequence_data)
  y_test_regression_list.append(sequence_target)
  row_cursor += 1

X_test = torch.stack(X_test_list, dim=0).to(torch.float)  # 테스트 데이터 텐서로 변환
y_test_regression = torch.tensor(y_test_regression_list, dtype=torch.float32) / y_normalizer

X_test -= (X_test - m) / s

print(X_test.shape, y_test_regression.shape)
# >>> torch.Size([24, 24, 18]) torch.Size([24])

# F 고찰내용
1. 데이터 로딩 및 변환
* NumPy와 PyTorch의 상호운용성: 데이터를 np.loadtxt로 불러오고, 이를 torch.from_numpy()를 사용하여 PyTorch 텐서로 변환할 수 있다.
* 특정 열 처리: converters를 사용하여 데이터를 로드할 때 특정 열의 값을 변환할 수 있다.
2. 슬라이딩 윈도우 기법
* 시계열 데이터에서 시퀀스 생성: 각 시퀀스가 과거 sequence_size 동안의 데이터를 포함하고, 그에 따라 하나의 타겟 값(자전거 대여 수)를 예측할 수 있다.
* 타겟과 시퀀스 데이터의 정렬: 시퀀스 데이터를 생성하고, 그 시퀀스의 마지막 값이 타겟 데이터로 설정되는 방식을 알 수 있다.
4. 데이터 정규화
* 훈련 데이터의 평균과 표준편차를 사용한 정규화: 훈련 데이터의 평균과 표준편차를 구해 이를 활용하여 훈련, 검증, 테스트 데이터를 정규화할 수 있다, 모델이 수렴하는 속도를 개선할 수 있고, 이상치에 민감하지 않게 할 수 있다.
* 데이터셋 간의 일관된 정규화: 모델이 과적합(overfitting)하지 않게 하고 일관된 성능을 유지할 수 있다.
5. 훈련, 검증, 테스트 데이터 분리
* 데이터 분할: 훈련, 검증, 테스트 데이터를 나누는 과정을 통해, 모델이 새로운 데이터를 얼마나 잘 예측할 수 있는지 평가할 수 있다.
* 슬라이딩 윈도우 기반 훈련 데이터 생성: 각 데이터셋에 대해 for 루프를 사용해 슬라이딩 윈도우 방식으로 데이터를 분리할 수 있다.
7. 모델 훈련을 위한 데이터 준비
* 시계열 데이터를 모델에 맞게 정렬: 시계열 데이터의 각 시간 단위 데이터를 하나의 샘플을 정규화하여 딥러닝 모델이 다룰 수 있는 형태로 바꿀 수 있다.


In [ ]:
import pandas as pd
from pathlib import Path
import os
import torch
import matplotlib.pyplot as plt

# BASE_PATH 설정: 프로젝트의 루트 경로를 기준으로 설정 (현재 파일의 상위 2개의 디렉토리)
BASE_PATH = str(Path("Deep_HW").resolve().parent.parent)    #코드 수정
import sys
sys.path.append(BASE_PATH)

# BTC_KRW 데이터 경로 설정 및 CSV 파일 읽기
btc_krw_path = os.path.join(BASE_PATH, "_00_data", "k_cryptocurrency", "BTC_KRW.csv")
df = pd.read_csv(btc_krw_path)  # CSV 파일을 DataFrame으로 로드
print(df)

# 데이터의 총 행(row) 크기 출력
row_size = len(df)
print("row_size:", row_size)

# 데이터의 컬럼 출력 (['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
columns = df.columns
print([column for column in columns])

# 'Date' 컬럼을 별도로 저장하고, DataFrame에서 제거
date_list = df['Date']
df = df.drop(columns=['Date'])

print(df)
print("#" * 100, 0)

#################################################################################################

# 시퀀스 크기, 검증 데이터 크기, 테스트 데이터 크기 설정
sequence_size = 10
validation_size = 100
test_size = 50

# 데이터 크기 계산: (총 행 크기 - 시퀀스 크기 + 1)
data_size = row_size - sequence_size + 1
print("data_size: {0}".format(data_size))

# 훈련, 검증, 테스트 데이터 크기 설정
train_size = data_size - (validation_size + test_size)
print("train_size: {0}, validation_size: {1}, test_size: {2}".format(train_size, validation_size, test_size))

print("#" * 100, 1)

#################################################################################################

# 훈련 데이터 준비
row_cursor = 0  # 데이터를 순차적으로 처리하기 위한 커서
y_normalizer = 1.0e7  # 가격 데이터를 정규화하기 위한 스케일링 값

# 훈련 데이터를 담을 리스트 초기화
X_train_list = []
y_train_regression_list = []
y_train_classification_list = []
y_train_date = []

# 훈련 데이터 생성 (시퀀스 기반)
for idx in range(0, train_size):
    # 시퀀스 데이터를 추출 (시퀀스 크기만큼의 데이터)
    sequence_data = df.iloc[idx: idx + sequence_size].values  # sequence_data.shape: (sequence_size, 5)
    X_train_list.append(torch.from_numpy(sequence_data))
    
    # 시퀀스의 마지막 값(Close)을 회귀 타겟으로 설정
    y_train_regression_list.append(df.iloc[idx + sequence_size - 1]["Close"])
    
    # 시퀀스의 마지막 두 값(Close)을 비교하여 상승/하락 여부를 분류 타겟으로 설정
    y_train_classification_list.append(
        1 if df.iloc[idx + sequence_size - 1]["Close"] >= df.iloc[idx + sequence_size - 2]["Close"] else 0
    )
    
    # 해당 시퀀스의 날짜 저장
    y_train_date.append(date_list[idx + sequence_size - 1])
    row_cursor += 1

# 리스트 형태의 데이터를 텐서로 변환
X_train = torch.stack(X_train_list, dim=0).to(torch.float)
y_train_regression = torch.tensor(y_train_regression_list, dtype=torch.float32) / y_normalizer  # 정규화된 회귀 타겟
y_train_classification = torch.tensor(y_train_classification_list, dtype=torch.int64)  # 분류 타겟
print(y_train_classification)

# 데이터 정규화 (평균, 표준편차)
m = X_train.mean(dim=0, keepdim=True)
s = X_train.std(dim=0, keepdim=True)
X_train -= m
X_train /= s
print(X_train.shape, y_train_regression.shape, y_train_classification.shape)
print("Label - Start Date: {0} ~ End Date: {1}".format(y_train_date[0], y_train_date[-1]))

print("#" * 100, 2)

#################################################################################################

# 검증 데이터 준비
X_validation_list = []
y_validation_regression_list = []
y_validation_classification_list = []
y_validation_date = []

# 검증 데이터 생성
for idx in range(row_cursor, row_cursor + validation_size):
    # 시퀀스 데이터를 추출
    sequence_data = df.iloc[idx: idx + sequence_size].values     # sequence_data.shape: (sequence_size, 5)
    X_validation_list.append(torch.from_numpy(sequence_data))
    
    # 시퀀스의 마지막 값(Close)을 회귀 타겟으로 설정
    y_validation_regression_list.append(df.iloc[idx + sequence_size - 1]["Close"])
    
    # 시퀀스의 마지막 두 값(Close)을 비교하여 상승/하락 여부를 분류 타겟으로 설정
    y_validation_classification_list.append(
        1 if df.iloc[idx + sequence_size - 1]["Close"] >= df.iloc[idx + sequence_size - 2]["Close"] else 0
    )
    
    # 해당 시퀀스의 날짜 저장
    y_validation_date.append(date_list[idx + sequence_size - 1])
    row_cursor += 1

# 리스트 데이터를 텐서로 변환
X_validation = torch.stack(X_validation_list, dim=0).to(torch.float)
y_validation_regression = torch.tensor(y_validation_regression_list, dtype=torch.float32) / y_normalizer  # 정규화된 회귀 타겟
y_validation_classification = torch.tensor(y_validation_classification_list, dtype=torch.int64)  # 분류 타겟
print(y_validation_classification)

# 검증 데이터 정규화 (훈련 데이터에서 구한 평균, 표준편차 사용)
X_validation = (X_validation - m) / s
print(X_validation.shape, y_validation_regression.shape, y_validation_classification.shape)
print("Label - Start Date: {0} ~ End Date: {1}".format(y_validation_date[0], y_validation_date[-1]))

print("#" * 100, 3)

#################################################################################################

# 테스트 데이터 준비
X_test_list = []
y_test_regression_list = []
y_test_classification_list = []
y_test_date = []

# 테스트 데이터 생성
for idx in range(row_cursor, row_cursor + test_size):
    # 시퀀스 데이터를 추출
    sequence_data = df.iloc[idx: idx + sequence_size].values   # sequence_data.shape: (sequence_size, 5)
    X_test_list.append(torch.from_numpy(sequence_data))
    
    # 시퀀스의 마지막 값(Close)을 회귀 타겟으로 설정
    y_test_regression_list.append(df.iloc[idx + sequence_size - 1]["Close"])
    
    # 시퀀스의 마지막 두 값(Close)을 비교하여 상승/하락 여부를 분류 타겟으로 설정
    y_test_classification_list.append(
        1 if df.iloc[idx + sequence_size - 1]["Close"] > df.iloc[idx + sequence_size - 2]["Close"] else 0
    )
    
    # 해당 시퀀스의 날짜 저장
    y_test_date.append(date_list[idx + sequence_size - 1])
    row_cursor += 1

# 리스트 데이터를 텐서로 변환
X_test = torch.stack(X_test_list, dim=0).to(torch.float)
y_test_regression = torch.tensor(y_test_regression_list, dtype=torch.float32) / y_normalizer  # 정규화된 회귀 타겟
y_test_classification = torch.tensor(y_test_classification_list, dtype=torch.int64)  # 분류 타겟
print(y_test_classification)

# 테스트 데이터 정규화 (훈련 데이터에서 구한 평균, 표준편차 사용)
X_test = (X_test - m) / s
print(X_test.shape, y_test_regression.shape, y_test_classification.shape)
print("Label - Start Date: {0} ~ End Date: {1}".format(y_test_date[0], y_test_date[-1]))

#######################################################################################

# 그래프 생성
fig, ax = plt.subplots(1, figsize=(13, 7))

# 훈련, 검증, 테스트 데이터의 회귀 타겟을 날짜별로 출력
ax.plot(y_train_date, y_train_regression * y_normalizer, label="y_train_regression", linewidth=2)
ax.plot(y_validation_date, y_validation_regression * y_normalizer, label="y_validation", linewidth=2)
ax.plot(y_test_date, y_test_regression * y_normalizer, label="y_test", linewidth=2)

# y축 레이블 설정
ax.set_ylabel('Bitcoin [KRW]', fontsize=14)

# x축 눈금 설정 (200 단위로 출력)
ax.set_xticks(ax.get_xticks()[::200])

# y축 숫자가 평범한 포맷으로 출력되도록 설정
plt.ticklabel_format(style='plain', axis='y')

# x축 레이블 회전
plt.xticks(rotation=25)

# 범례 설정
ax.legend(loc='upper left', fontsize=16)

# 그래프 출력
plt.show()

# G 고찰내용
1. DF로 데이터를 다룰 수 있다.
2. PyTorch 텐서 처리: 데이터를 PyTorch 텐서로 변환하고 이를 모델에 입력할 수 있는 형식으로 준비할 수 있다.
3. 데이터셋 시각화: 훈련, 검증, 테스트 데이터를 시각적으로 표현할 수 있다.


In [ ]:
import torch
import os
import scipy.io.wavfile as wavfile
#경로수정
audio_1_path = os.path.join(os.path.pardir, "_00_data", "f_audio-chirp", "1-100038-A-14.wav")
audio_2_path = os.path.join(os.path.pardir, "_00_data", "f_audio-chirp", "1-100210-A-36.wav")

freq_1, waveform_arr_1 = wavfile.read(audio_1_path) # 첫 번째 오디오 파일 읽기
print(freq_1)   # 샘플링 주파수 출력
print(type(waveform_arr_1)) # 오디오 데이터의 타입 확인 (numpy 배열)
print(len(waveform_arr_1))  # 오디오 데이터의 길이 출력 (샘플 수)
print(waveform_arr_1)       # 오디오 데이터 값 출력

freq_2, waveform_arr_2 = wavfile.read(audio_2_path)

# 두 개의 오디오 데이터를 하나의 텐서로 결합 (형상: [2, 1, 220,500])
# 2개의 오디오, 1채널, 각 오디오 데이터는 220,500 샘플
waveform = torch.empty(2, 1, 220_500)
# 첫 번째 오디오 데이터를 텐서로 변환 후 저장
waveform[0, 0] = torch.from_numpy(waveform_arr_1).float()
waveform[1, 0] = torch.from_numpy(waveform_arr_2).float()

print(waveform.shape)   # 텐서 크기 출력 (형상: [2, 1, 220500])

print("#" * 50, 1)

from scipy import signal

_, _, sp_arr_1 = signal.spectrogram(waveform_arr_1, freq_1) # 첫 번째 오디오 데이터에 대한 스펙트로그램 계산 (주파수 축, 시간 축, 스펙트럼 값 반환)
_, _, sp_arr_2 = signal.spectrogram(waveform_arr_2, freq_2)

# 스펙트로그램을 텐서로 변환
sp_1 = torch.from_numpy(sp_arr_1)
sp_2 = torch.from_numpy(sp_arr_2)
print(sp_1.shape)
print(sp_2.shape)

# 왼쪽 채널과 오른쪽 채널을 의미하는 텐서로 저장 (스테레오 형태)
sp_left_t = torch.from_numpy(sp_arr_1)
sp_right_t = torch.from_numpy(sp_arr_2)
print(sp_left_t.shape)
print(sp_right_t.shape)

# 두 채널을 하나의 텐서로 스택 (0번째 축에 추가), 차원 확장
sp_t = torch.stack((sp_left_t, sp_right_t), dim=0).unsqueeze(dim=0)
print(sp_t.shape)


# H 고찰내용
1. 오디오 데이터의 로드와 처리
* scipy.io.wavfile.read()함수로 오디오 데이터를 불러올 수 있다.
* 오디오 파일은 일반적으로 numpy 배열 형태로 로드된다.
2. PyTorch에서 오디오 데이터를 다루는 방식
* 오디오 데이터를 torch.from_numpy()를 사용하여 numpy 배열을 텐서로 변환, .float()를 통해 실수형(float) 데이터 타입으로 변환한다.
3. scipy.signal.spectrogram() 함수를 사용하여 오디오 데이터를 주파수 및 시간 축으로 변환하여 스펙트로그램을 계산할 수 있다.
4. torch.from_numpy()를 사용해 스펙트로그램을 텐서로 변환한 후, torch.stack()을 이용해 두 스펙트로그램을 하나의 텐서로 결합할 수 있다.

In [ ]:
# pip install imageio[ffmpeg]
import torch
import os
import imageio
#경로수정
video_path = os.path.join(os.path.pardir, "_00_data", "g_video-cockatoo", "cockatoo.mp4")

# imageio 라이브러리를 통해 비디오 파일을 읽기 위한 reader 객체 생성
reader = imageio.get_reader(video_path)
print(type(reader))
# 비디오 파일의 메타데이터 출력 (프레임 크기, 프레임 수, 비디오 포맷 등)
meta = reader.get_meta_data()
print(meta)

# 비디오 프레임을 하나씩 순회하며 처리
for i, frame in enumerate(reader):
    # 각 프레임을 numpy 배열로부터 torch 텐서로 변환 후 float 타입으로 변환
    frame = torch.from_numpy(frame).float()  # frame.shape: [360, 480, 3]
    print(i, frame.shape)   # i, torch.Size([360, 480, 3])

# 비디오 데이터의 초기 정보 설정
n_channels = 3  # 비디오 데이터의 채널 수 (RGB이므로 3채널)
n_frames = 529  # 비디오 파일의 전체 프레임 수 (메타데이터에서 얻은 값)
video = torch.empty(1, n_frames, n_channels, *meta['size'])  # (1, 529, 3, 480, 360)
print(video.shape)

# 다시 비디오 파일을 읽어 각 프레임을 텐서에 저장
for i, frame in enumerate(reader):
    # 각 프레임을 numpy 배열로부터 torch 텐서로 변환 후 float 타입으로 변환
    frame = torch.from_numpy(frame).float()       # frame.shape: [360, 480, 3]
    # torch 텐서의 차원을 순서를 변경하여 (C, W, H) 형태로 변환 (3채널, 480너비, 360높이)
    frame = torch.permute(frame, dims=(2, 1, 0))  # frame.shape: [3, 480, 360]
    # 해당 프레임을 비디오 텐서의 i번째 인덱스에 저장
    video[0, i] = frame

# 비디오 텐서의 차원 순서를 변경 (shape을 [1, 3, 529, 480, 360]으로 변환하여 PyTorch에서 사용하기 쉽게 변환)
video = video.permute(dims=(0, 2, 1, 3, 4))
print(video.shape)


# I 고찰내용
1. imageio.get_reader()를 통해 비디오 파일을 읽을 수 있다.
2. get_meta_data()로 비디오 파일의 크기, 프레임 수 등 중요한 정보를 얻을 수 있다.
3. 비디오 프레임을 numpy 배열에서 torch 텐서로 변환하고, 적절한 데이터 타입(float)으로 변환할 수 있다.
4. torch.permute()로 채널, 너비, 높이의 차원을 변경해, CNN과 같은 모델에서 사용할 수 있는 형식으로 변환할 수 있다.

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split


class LinearRegressionDataset(Dataset):
  def __init__(self, N=50, m=-3, b=2, *args, **kwargs):
    # N: 샘플 수, 기본값은 50
    # m: 기울기 (slope), 기본값은 -3
    # b: y절편 (offset), 기본값은 2
    super().__init__(*args, **kwargs)
    # N개의 2차원 무작위 입력 데이터 x를 생성합니다.
    self.x = torch.rand(N, 2)
    self.noise = torch.rand(N) * 0.2    # 무작위 노이즈를 추가하여 데이터가 더 현실적이게 만듭니다.
    self.m = m
    self.b = b
    self.y = (torch.sum(self.x * self.m) + self.b + self.noise).unsqueeze(-1)   # y값을 생성: y = m * x + b + noise

  def __len__(self):
    return len(self.x)

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
      len(self.x), self.x.shape, self.y.shape
    )
    return str


if __name__ == "__main__":
    # LinearRegressionDataset 클래스를 이용하여 데이터셋을 생성합니다
    linear_regression_dataset = LinearRegressionDataset()

    print(linear_regression_dataset)    # 데이터셋 정보 출력

    print("#" * 50, 1)
    # 데이터셋의 각 샘플(입력 데이터와 타겟 값)을 출력합니다.
    for idx, sample in enumerate(linear_regression_dataset):
        input, target = sample
        print("{0} - {1}: {2}".format(idx, input, target))
    # 데이터셋을 훈련, 검증, 테스트 세트로 나눕니다 (비율: 70%, 20%, 10%).
    train_dataset, validation_dataset, test_dataset = random_split(linear_regression_dataset, [0.7, 0.2, 0.1])

    print("#" * 50, 2)

    print(len(train_dataset), len(validation_dataset), len(test_dataset))

    print("#" * 50, 3)
    
    train_data_loader = DataLoader(
        # 훈련 데이터셋을 DataLoader에 넣어 배치로 묶고 셔플링을 수행합니다.
        dataset=train_dataset,
        batch_size=4,
        shuffle=True
    )

    for idx, batch in enumerate(train_data_loader):
        # DataLoader를 이용해 배치 단위로 데이터를 출력합니다.
        input, target = batch
        print("{0} - {1}: {2}".format(idx, input, target))


# J 고찰내용
1. PyTorch Dataset 클래스와 커스텀 데이터셋 생성
* Dataset 클래스를 상속받아 커스텀 데이터셋을 정의할 수 있다.
*  __getitem__은 주어진 인덱스에 해당하는 데이터를 반환하며, __len__은 데이터셋의 총 길이를 반환한다.
* 객체 정보를 사람이 읽기 좋은 형태로 출력할 수 있도록 __str__ 메서드를 오버라이딩할 수 있다.
2. 데이터 생성
* torch.rand()를 이용해 2차원 랜덤 데이터를 생성할 수 있다.
* self.noise = torch.rand(N) * 0.2와 같은 코드를 통해 모델이 더 현실적인 데이터에 대한 일반화를 학습하도록 노이즈를 추가할 수 있다.
* y = m * x + b 형태의 데이터를 생성하여, 선형 회귀에서 사용될 데이터셋을 직접 정의할 수 있다.
3. DataLoader를 사용해 데이터셋을 배치 단위로 처리할 수 있다.
4. random_split 함수를 사용해 데이터를 훈련, 검증, 테스트 세트로 나눌 수 있다.


In [1]:
import os
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms


class DogCat2DImageDataset(Dataset):
  def __init__(self):
    # 이미지 전처리: 이미지를 (256, 256) 크기로 리사이즈한 후 텐서로 변환
    self.image_transforms = transforms.Compose([
      transforms.Resize(size=(256, 256)),  # 이미지 크기를 256x256으로 리사이즈
      transforms.ToTensor()  # 이미지를 텐서로 변환
    ])

    # 강아지와 고양이 이미지 파일 경로 설정
    dogs_dir = os.path.join(os.path.pardir, "_00_data", "a_image-dog")
    cats_dir = os.path.join(os.path.pardir, "_00_data", "b_image-cats")

    # 이미지 파일을 열고 리스트에 저장
    image_lst = [
      Image.open(os.path.join(dogs_dir, "bobby.jpg")),  # 강아지 이미지 (1280x720 크기)
      Image.open(os.path.join(cats_dir, "cat1.png")),   # 고양이 이미지 1 (256x256 크기)
      Image.open(os.path.join(cats_dir, "cat2.png")),   # 고양이 이미지 2 (256x256 크기)
      Image.open(os.path.join(cats_dir, "cat3.png"))    # 고양이 이미지 3 (256x256 크기)
    ]

    # 이미지 전처리 적용 및 텐서화 후 스택하여 하나의 텐서로 결합 (형상: [4, 3, 256, 256])
    image_lst = [self.image_transforms(img) for img in image_lst]
    self.images = torch.stack(image_lst, dim=0)

    # 이미지 레이블 설정: 0은 강아지, 1은 고양이 (형상: [4, 1])
    self.image_labels = torch.tensor([[0], [1], [1], [1]])

  # 데이터셋 크기를 반환하는 함수
  def __len__(self):
    return len(self.images)

  # 인덱스를 기반으로 이미지와 해당 레이블을 반환하는 함수
  def __getitem__(self, idx):
    return self.images[idx], self.image_labels[idx]

  # 데이터셋 정보(크기, 입력 형상, 레이블 형상)를 문자열로 반환하는 함수
  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
      len(self.images), self.images.shape, self.image_labels.shape
    )
    return str


# 메인 실행 부분
if __name__ == "__main__":
  # DogCat2DImageDataset 객체 생성
  dog_cat_2d_image_dataset = DogCat2DImageDataset()

  # 데이터셋 정보 출력
  print(dog_cat_2d_image_dataset)

  print("#" * 50, 1)

  # 데이터셋 내의 각 샘플에 대해 입력 데이터와 레이블 출력
  for idx, sample in enumerate(dog_cat_2d_image_dataset):
    input, target = sample
    print("{0} - {1}: {2}".format(idx, input.shape, target))

  # 데이터셋을 70%는 학습 데이터, 30%는 테스트 데이터로 분할
  train_dataset, test_dataset = random_split(dog_cat_2d_image_dataset, [0.7, 0.3])

  print("#" * 50, 2)

  # 학습 데이터셋과 테스트 데이터셋의 크기 출력
  print(len(train_dataset), len(test_dataset))

  print("#" * 50, 3)

  # DataLoader를 사용하여 학습 데이터셋을 배치 크기 2로 로드
  train_data_loader = DataLoader(
    dataset=train_dataset,
    batch_size=2,  # 배치 크기 2
    shuffle=True   # 데이터를 섞어서 로드
  )

  # 각 배치에 대해 입력 데이터와 레이블 출력
  for idx, batch in enumerate(train_data_loader):
    input, target = batch
    print("{0} - {1}: {2}".format(idx, input.shape, target))


Data Size: 4, Input Shape: torch.Size([4, 3, 256, 256]), Target Shape: torch.Size([4, 1])
################################################## 1
0 - torch.Size([3, 256, 256]): tensor([0])
1 - torch.Size([3, 256, 256]): tensor([1])
2 - torch.Size([3, 256, 256]): tensor([1])
3 - torch.Size([3, 256, 256]): tensor([1])
################################################## 2
3 1
################################################## 3
0 - torch.Size([2, 3, 256, 256]): tensor([[1],
        [1]])
1 - torch.Size([1, 3, 256, 256]): tensor([[1]])


# K 고찰내용
1. torch패키지를 사용할 시 다른 패키지와 버전 충돌이 일어나지 않도록 설치해야함.


In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split


class WineDataset(Dataset):
    def __init__(self):
        # 데이터 파일 경로 설정
        wine_path = os.path.join(os.path.pardir, "_00_data", "d_tabular-wine", "winequality-white.csv") # 경로수정
        
        # CSV 파일을 NumPy 배열로 로드
        wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";", skiprows=1)
        
        # NumPy 배열을 PyTorch 텐서로 변환
        wineq = torch.from_numpy(wineq_numpy)

        # 데이터와 타겟 분리
        data = wineq[:, :-1]  # 마지막 열을 제외한 모든 열을 데이터로 사용
        target = wineq[:, -1].long()  # 마지막 열을 타겟으로 사용 (정수형으로 변환)
        
        # 데이터 정규화 (평균과 분산을 사용)
        data_mean = torch.mean(data, dim=0)
        data_var = torch.var(data, dim=0)
        self.data = (data - data_mean) / torch.sqrt(data_var)
        
        # 타겟을 원-핫 인코딩으로 변환
        eye_matrix = torch.eye(10)  # 10개의 클래스에 대한 단위 행렬 생성
        self.target = eye_matrix[target]  # 타겟 값에 해당하는 행을 선택하여 원-핫 인코딩 생성

        # 데이터와 타겟의 길이가 같은지 확인
        assert len(self.data) == len(self.target)

    def __len__(self):
        # 데이터셋의 크기 반환
        return len(self.data)

    def __getitem__(self, idx):
        # 주어진 인덱스에 대한 데이터와 타겟을 반환
        wine_feature = self.data[idx]
        wine_target = self.target[idx]
        return wine_feature, wine_target

    def __str__(self):
        # 데이터셋의 크기와 입력 및 타겟의 형상 정보 반환
        str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
            len(self.data), self.data.shape, self.target.shape
        )
        return str


if __name__ == "__main__":
    # WineDataset 객체 생성
    wine_dataset = WineDataset()

    # 데이터셋 정보 출력
    print(wine_dataset)

    print("#" * 50, 1)

    # 데이터셋 내의 각 샘플에 대해 입력 데이터와 타겟 출력
    for idx, sample in enumerate(wine_dataset):
        input, target = sample
        print("{0} - {1}: {2}".format(idx, input.shape, target.shape))

    # 데이터셋을 70%는 학습 데이터, 20%는 검증 데이터, 10%는 테스트 데이터로 분할
    train_dataset, validation_dataset, test_dataset = random_split(wine_dataset, [0.7, 0.2, 0.1])

    print("#" * 50, 2)

    # 학습 데이터셋, 검증 데이터셋, 테스트 데이터셋의 크기 출력
    print(len(train_dataset), len(validation_dataset), len(test_dataset))

    print("#" * 50, 3)

    # DataLoader를 사용하여 학습 데이터셋을 배치 크기 32로 로드
    train_data_loader = DataLoader(
        dataset=train_dataset,
        batch_size=32,  # 배치 크기 32
        shuffle=True,  # 데이터를 섞어서 로드
        drop_last=True  # 배치 크기가 데이터셋의 크기로 나누어떨어지지 않는 경우 마지막 배치를 버림
    )

    # 각 배치에 대해 입력 데이터와 타겟 출력
    for idx, batch in enumerate(train_data_loader):
        input, target = batch
        print("{0} - {1}: {2}".format(idx, input.shape, target.shape))

# I 고찰내용
1. PyTorch의 Dataset 클래스를 상속하여 데이터셋을 정의할 수 있다.
2. 배치 크기를 설정하고, 데이터를 섞어서(shuffle=True) 학습하는 과정에서 데이터의 마지막 배치를 버리는 옵션(drop_last=True)을 사용할 수 있다.

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split

class CaliforniaHousingDataset(Dataset):
    def __init__(self):
        # scikit-learn의 California housing 데이터셋을 로드
        from sklearn.datasets import fetch_california_housing
        housing = fetch_california_housing()

        # 데이터 정규화: 평균과 분산을 사용하여 데이터 정규화
        data_mean = np.mean(housing.data, axis=0)  # 각 특성의 평균
        data_var = np.var(housing.data, axis=0)    # 각 특성의 분산
        # 데이터 정규화 (평균 0, 분산 1로 변환)
        self.data = torch.tensor((housing.data - data_mean) / np.sqrt(data_var), dtype=torch.float32)
        
        # 타겟 변수 변환: 텐서로 변환하고 차원을 추가 (모델 학습을 위해 2D 텐서로 변환)
        self.target = torch.tensor(housing.target, dtype=torch.float32).unsqueeze(dim=-1)

    def __len__(self):
        # 데이터셋의 크기 반환
        return len(self.data)

    def __getitem__(self, idx):
        # 주어진 인덱스에 대해 데이터와 타겟을 반환
        sample_data = self.data[idx]
        sample_target = self.target[idx]
        return sample_data, sample_target

    def __str__(self):
        # 데이터셋의 크기와 입력 및 타겟의 형상 정보를 문자열로 반환
        return "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
            len(self.data), self.data.shape, self.target.shape
        )

if __name__ == "__main__":
    # CaliforniaHousingDataset 객체 생성
    california_housing_dataset = CaliforniaHousingDataset()

    # 데이터셋 정보 출력
    print(california_housing_dataset)

    print("#" * 50, 1)

    # 데이터셋 내의 각 샘플에 대해 입력 데이터와 타겟 출력
    for idx, sample in enumerate(california_housing_dataset):
        input, target = sample
        print("{0} - {1}: {2}".format(idx, input.shape, target.shape))

    # 데이터셋을 70%는 학습 데이터, 20%는 검증 데이터, 10%는 테스트 데이터로 분할
    train_dataset, validation_dataset, test_dataset = random_split(california_housing_dataset, [0.7, 0.2, 0.1])

    print("#" * 50, 2)

    # 학습 데이터셋, 검증 데이터셋, 테스트 데이터셋의 크기 출력
    print(len(train_dataset), len(validation_dataset), len(test_dataset))

    print("#" * 50, 3)

    # DataLoader를 사용하여 학습 데이터셋을 배치 크기 32로 로드
    train_data_loader = DataLoader(
        dataset=train_dataset,
        batch_size=32,  # 배치 크기 32
        shuffle=True,  # 데이터를 섞어서 로드
        drop_last=True  # 배치 크기가 데이터셋의 크기로 나누어떨어지지 않는 경우 마지막 배치를 버림
    )

    # 각 배치에 대해 입력 데이터와 타겟 출력
    for idx, batch in enumerate(train_data_loader):
        input, target = batch
        print("{0} - {1}: {2}".format(idx, input.shape, target.shape))

# M 고찰내용
1. 데이터 셋을 정의하고 정규화 후 배치처리하는 모델 학습 준비의 과정을 알 수 있다.

In [ ]:
import os
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split


BASE_PATH = str(Path("Deep_HW").resolve().parent.parent)  
import sys
sys.path.append(BASE_PATH)

# BikesDataset 클래스 정의
class BikesDataset(Dataset):
    def __init__(self, train=True, test_days=1):
        self.train = train  # 학습용 데이터셋인지 테스트용 데이터셋인지 구분하는 플래그
        self.test_days = test_days  # 테스트 데이터로 사용할 일수

        # 데이터셋 파일 경로 설정
        bikes_path = os.path.join(BASE_PATH, "_00_data", "e_time-series-bike-sharing-dataset", "hour-fixed.csv")

        # CSV 파일을 NumPy 배열로 불러오기 (converters를 사용하여 날짜 컬럼에서 일(day)만 추출)
        bikes_numpy = np.loadtxt(
            fname=bikes_path, dtype=np.float32, delimiter=",", skiprows=1,
            converters={
                1: lambda x: float(x[8:10])  # 2011-01-07 --> 07 --> 7
            }
        )
        bikes = torch.from_numpy(bikes_numpy)  # NumPy 배열을 PyTorch 텐서로 변환

        # 데이터를 일(day) 단위로 나누기 (24시간씩 그룹화)
        daily_bikes = bikes.view(-1, 24, bikes.shape[1])  # daily_bikes.shape: torch.Size([730, 24, 17])
        self.daily_bikes_target = daily_bikes[:, :, -1].unsqueeze(dim=-1)  # 목표(target) 값 설정

        self.daily_bikes_data = daily_bikes[:, :, :-1]  # 마지막 열을 제외한 나머지를 입력 데이터로 사용
        eye_matrix = torch.eye(4)  # 날씨 데이터를 원-핫 인코딩할 때 사용할 단위 행렬 생성

        # 날씨 정보를 원-핫 인코딩하여 입력 데이터에 추가
        day_data_torch_list = []
        for daily_idx in range(self.daily_bikes_data.shape[0]):  # range(730)
            day = self.daily_bikes_data[daily_idx]  # day.shape: [24, 17]
            weather_onehot = eye_matrix[day[:, 9].long() - 1]  # 날씨 데이터 원-핫 인코딩
            day_data_torch = torch.cat(tensors=(day, weather_onehot), dim=1)  # 입력 데이터와 원-핫 인코딩된 데이터를 병합
            day_data_torch_list.append(day_data_torch)

        self.daily_bikes_data = torch.stack(day_data_torch_list, dim=0)  # 리스트를 텐서로 변환

        # 데이터를 분리하여 특정 열을 제외
        self.daily_bikes_data = torch.cat(
            [self.daily_bikes_data[:, :, :9], self.daily_bikes_data[:, :, 10:]], dim=2
        )

        total_length = len(self.daily_bikes_data)  # 전체 데이터 길이
        self.train_bikes_data = self.daily_bikes_data[:total_length - test_days]  # 학습 데이터
        self.train_bikes_targets = self.daily_bikes_target[:total_length - test_days]  # 학습 데이터의 목표값
        train_temperatures = self.train_bikes_data[:, :, 9]  # 학습 데이터에서 온도 정보 추출
        train_temperatures_mean = torch.mean(train_temperatures)  # 온도의 평균 계산
        train_temperatures_std = torch.std(train_temperatures)  # 온도의 표준편차 계산

        # 학습 데이터의 온도 정보를 정규화
        self.train_bikes_data[:, :, 9] = \
            (self.train_bikes_data[:, :, 9] - train_temperatures_mean) / train_temperatures_std

        # 테스트 데이터가 필요한 경우, 테스트 데이터 설정 및 정규화
        if not train:
            self.test_bikes_data = self.daily_bikes_data[-test_days:]  # 테스트 데이터
            self.test_bikes_targets = self.daily_bikes_target[-test_days:]  # 테스트 데이터의 목표값
            self.test_bikes_data[:, :, 9] = \
                (self.test_bikes_data[:, :, 9] - train_temperatures_mean) / train_temperatures_std  # 온도 정규화

    # 데이터셋의 크기 반환
    def __len__(self):
        return len(self.train_bikes_data) if self.train is True else len(self.test_bikes_data)

    # 주어진 인덱스에 해당하는 입력 데이터와 목표값 반환
    def __getitem__(self, idx):
        bike_feature = self.train_bikes_data[idx] if self.train is True else self.test_bikes_data[idx]
        bike_target = self.train_bikes_targets[idx] if self.train is True else self.test_bikes_targets[idx]
        return bike_feature, bike_target


if __name__ == "__main__":
    # 학습 데이터셋 생성
    train_bikes_dataset = BikesDataset(train=True, test_days=1)
    print(train_bikes_dataset)

    print("#" * 50, 1)

    # 학습 데이터셋을 80% 학습 데이터와 20% 검증 데이터로 분할
    train_dataset, validation_dataset = random_split(train_bikes_dataset, [0.8, 0.2])

    # 학습 데이터셋 출력
    print("[TRAIN]")
    for idx, sample in enumerate(train_dataset):
        input, target = sample
        print("{0} - {1}: {2}".format(idx, input.shape, target.shape))

    # 학습 데이터를 DataLoader로 배치 단위로 로드
    train_data_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True, drop_last=True)

    # 배치 단위로 데이터 출력
    for idx, batch in enumerate(train_data_loader):
        input, target = batch
        print("{0} - {1}: {2}".format(idx, input.shape, target.shape))

    print("#" * 50, 2)

    # 검증 데이터셋 출력
    print("[VALIDATION]")
    for idx, sample in enumerate(validation_dataset):
        input, target = sample
        print("{0} - {1}: {2}".format(idx, input.shape, target.shape))

    # 검증 데이터를 DataLoader로 로드
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=32)

    # 배치 단위로 검증 데이터 출력
    for idx, batch in enumerate(validation_data_loader):
        input, target = batch
        print("{0} - {1}: {2}".format(idx, input.shape, target.shape))

    print("#" * 50, 3)

    # 테스트 데이터셋 생성
    test_dataset = BikesDataset(train=False, test_days=1)
    print(test_dataset)

    # 테스트 데이터 출력
    print("[TEST]")
    for idx, sample in enumerate(test_dataset):
        input, target = sample
        print("{0} - {1}: {2}".format(idx, input.shape, target.shape))

    # 테스트 데이터를 DataLoader로 로드
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

    # 테스트 데이터 출력
    for idx, batch in enumerate(test_data_loader):
        input, target = batch
        print("{0} - {1}: {2}".format(idx, input.shape, target.shape))


# N 고찰 내용
1. PyTorch Dataset 및 DataLoader 활용할 수 있다.
2. np.loadtxt의 converters 인자를 사용하여, 날짜 데이터를 처리할 때 2011-01-07 형식에서 07만 추출하는 커스텀 변환기를 적용할 수 있다.
3. 데이터를 재구성할 때 사용하는 view 메서드는 특정 차원으로 데이터를 변환하는 데 유용하다.

In [ ]:
import os
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split

# 프로젝트 디렉토리 경로 설정
BASE_PATH = str(Path("Deep_HW").resolve().parent.parent) 
import sys
sys.path.append(BASE_PATH)

# 자전거 대여 데이터셋을 위한 Dataset 클래스 정의
class HourlyBikesDataset(Dataset):
    def __init__(self, X, y):
        # 입력 데이터(X)와 타겟 데이터(y)를 받아서 초기화
        self.X = X
        self.y = y

        # X와 y의 크기가 같은지 확인
        assert len(self.X) == len(self.y)

    def __len__(self):
        # 데이터셋의 크기 반환
        return len(self.X)

    def __getitem__(self, idx):
        # 주어진 인덱스(idx)에 해당하는 입력 데이터와 타겟 데이터 반환
        X = self.X[idx]
        y = self.y[idx]
        return X, y

    def __str__(self):
        # 데이터셋 정보(크기, 입력 데이터 모양, 타겟 데이터 모양)를 문자열로 반환
        str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
            len(self.X), self.X.shape, self.y.shape
        )
        return str

# 자전거 대여 데이터를 불러오고 학습, 검증, 테스트용 데이터셋을 만드는 함수 정의
def get_hourly_bikes_data(sequence_size=24, validation_size=96, test_size=24, y_normalizer=100):
    # 자전거 대여 데이터 파일 경로
    bikes_path = os.path.join(BASE_PATH, "_00_data", "e_time-series-bike-sharing-dataset", "hour-fixed.csv")

    # CSV 파일을 불러와 NumPy 배열로 변환 (1번 컬럼을 날짜로 변환하여 특정 형식에 맞춤)
    bikes_numpy = np.loadtxt(
        fname=bikes_path, dtype=np.float32, delimiter=",", skiprows=1,
        converters={
            1: lambda x: float(x[8:10])  # 2011-01-07 --> 07 --> 7
        }
    )

    # NumPy 배열을 PyTorch 텐서로 변환하고, 마지막 열('cnt' 열)을 타겟 데이터로 분리
    bikes_data = torch.from_numpy(bikes_numpy).to(torch.float) # >>> torch.Size([17520, 17])
    bikes_target = bikes_data[:, -1].unsqueeze(dim=-1)  # 'cnt' 열을 타겟으로 사용
    bikes_data = bikes_data[:, :-1]  # 마지막 열을 제외한 나머지를 입력 데이터로 사용

    # 날씨 데이터를 원-핫 인코딩하기 위한 단위 행렬
    eye_matrix = torch.eye(4)

    # 날씨 정보를 원-핫 인코딩하여 데이터를 변환
    data_torch_list = []
    for idx in range(bikes_data.shape[0]):
        hour_data = bikes_data[idx]  # 각 시간 데이터
        weather_onehot = eye_matrix[hour_data[9].long() - 1]  # 날씨 정보를 원-핫 인코딩
        concat_data_torch = torch.cat(tensors=(hour_data, weather_onehot), dim=-1)  # 데이터와 원-핫 인코딩 데이터를 병합
        data_torch_list.append(concat_data_torch)

    # 리스트를 텐서로 변환하여 새로운 데이터셋으로 만듦
    bikes_data = torch.stack(data_torch_list, dim=0)
    # 불필요한 열(시간 및 원래 날씨 열)을 제거하여 최종 입력 데이터 구성
    bikes_data = torch.cat([bikes_data[:, 1:9], bikes_data[:, 10:]], dim=-1)
    print(bikes_data.shape, "!!!")  # >>> torch.Size([17520, 18])

    # 학습, 검증, 테스트 데이터셋의 크기 계산
    data_size = len(bikes_data) - sequence_size
    train_size = data_size - (validation_size + test_size)

    # 학습 데이터셋 구성
    row_cursor = 0
    X_train_list = []
    y_train_regression_list = []
    for idx in range(0, train_size):
        sequence_data = bikes_data[idx: idx + sequence_size]  # 입력 데이터 시퀀스
        sequence_target = bikes_target[idx + sequence_size - 1]  # 해당 시퀀스의 타겟 값
        X_train_list.append(sequence_data)
        y_train_regression_list.append(sequence_target)
        row_cursor += 1

    # 학습 데이터 텐서로 변환
    X_train = torch.stack(X_train_list, dim=0).to(torch.float)
    y_train_regression = torch.tensor(y_train_regression_list, dtype=torch.float32) / y_normalizer

    # 학습 데이터 정규화 (평균과 표준편차를 사용)
    m = X_train.mean(dim=0, keepdim=True)
    s = X_train.std(dim=0, keepdim=True)
    X_train = (X_train - m) / s

    # 검증 데이터셋 구성
    X_validation_list = []
    y_validation_regression_list = []
    for idx in range(row_cursor, row_cursor + validation_size):
        sequence_data = bikes_data[idx: idx + sequence_size]
        sequence_target = bikes_target[idx + sequence_size - 1]
        X_validation_list.append(sequence_data)
        y_validation_regression_list.append(sequence_target)
        row_cursor += 1

    # 검증 데이터 텐서로 변환 및 정규화
    X_validation = torch.stack(X_validation_list, dim=0).to(torch.float)
    y_validation_regression = torch.tensor(y_validation_regression_list, dtype=torch.float32) / y_normalizer
    X_validation -= m
    X_validation /= s

    # 테스트 데이터셋 구성
    X_test_list = []
    y_test_regression_list = []
    for idx in range(row_cursor, row_cursor + test_size):
        sequence_data = bikes_data[idx: idx + sequence_size]
        sequence_target = bikes_target[idx + sequence_size - 1]
        X_test_list.append(sequence_data)
        y_test_regression_list.append(sequence_target)
        row_cursor += 1

    # 테스트 데이터 텐서로 변환 및 정규화
    X_test = torch.stack(X_test_list, dim=0).to(torch.float)
    y_test_regression = torch.tensor(y_test_regression_list, dtype=torch.float32) / y_normalizer
    X_test -= m
    X_test /= s

    # 학습, 검증, 테스트 데이터셋 반환
    return (
        X_train, X_validation, X_test,
        y_train_regression, y_validation_regression, y_test_regression
    )


# 메인 코드: 데이터셋을 불러와서 DataLoader로 구성하고 출력
if __name__ == "__main__":
    X_train, X_validation, X_test, y_train, y_validation, y_test = get_hourly_bikes_data(
        sequence_size=24, validation_size=96, test_size=24, y_normalizer=100
    )

    print("Train: {0}, Validation: {1}, Test: {2}".format(len(X_train), len(X_validation), len(X_test)))

    # 학습, 검증, 테스트 데이터셋을 각각 HourlyBikesDataset으로 래핑
    train_hourly_bikes_dataset = HourlyBikesDataset(X=X_train, y=y_train)
    validation_hourly_bikes_dataset = HourlyBikesDataset(X=X_validation, y=y_validation)
    test_houly_bikes_dataset = HourlyBikesDataset(X=X_test, y=y_test)

    # 학습 데이터셋을 DataLoader로 로드하여 배치 단위로 처리
    train_data_loader = DataLoader(
        dataset=train_hourly_bikes_dataset, batch_size=32, shuffle=True, drop_last=True
    )

    # 데이터 배치 출력 코드
    # for idx, batch in enumerate(train_data_loader):
    #     input, target = batch
    #     print("{0} - {1}: {2}, {3}".format(idx, input.shape, target.shape, target))


# O 고찰내용
#### dataset을 활용해서 생기는 이점
1. 효율적인 메모리 사용
2. 학습 성능 향상
3. 셔플링(Shuffling)
4. 드롭 아웃 기능
5. 유연한 배치 크기 설정
6. 병렬 데이터 로드

In [ ]:
# https://towardsdatascience.com/cryptocurrency-price-prediction-using-deep-learning-70cfca50dd3a
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import os
import torch
import pandas as pd
import numpy as np

BASE_PATH = str(Path("Deep_HW").resolve().parent.parent) 
import sys
sys.path.append(BASE_PATH)


class CryptoCurrencyDataset(Dataset):
  def __init__(self, X, y, is_regression=True):
    self.X = X
    self.y = y

    assert len(self.X) == len(self.y)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    X = self.X[idx]
    y = self.y[idx]
    return X, y

  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
      len(self.X), self.X.shape, self.y.shape
    )
    return str


def get_cryptocurrency_data(
    sequence_size=10, validation_size=100, test_size=10, target_column='Close', y_normalizer=1.0e7, is_regression=True
):
  btc_krw_path = os.path.join(BASE_PATH, "_00_data", "k_cryptocurrency", "BTC_KRW.csv")
  df = pd.read_csv(btc_krw_path)
  row_size = len(df)
  # ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
  date_list = df['Date']

  df = df.drop(columns=['Date'])

  data_size = row_size - sequence_size
  train_size = data_size - (validation_size + test_size)
  #################################################################################################

  row_cursor = 0

  X_train_list = []
  y_train_regression_list = []
  y_train_classification_list = []
  y_train_date = []
  for idx in range(0, train_size):
    sequence_data = df.iloc[idx: idx + sequence_size].values  # sequence_data.shape: (sequence_size, 5)
    X_train_list.append(torch.from_numpy(sequence_data))
    y_train_regression_list.append(df.iloc[idx + sequence_size][target_column])
    y_train_classification_list.append(
      1 if df.iloc[idx + sequence_size][target_column] >= df.iloc[idx + sequence_size - 1][target_column] else 0
    )
    y_train_date.append(date_list[idx + sequence_size])
    row_cursor += 1

  X_train = torch.stack(X_train_list, dim=0).to(torch.float)
  y_train_regression = torch.tensor(y_train_regression_list, dtype=torch.float32) / y_normalizer
  y_train_classification = torch.tensor(y_train_classification_list, dtype=torch.int64)

  m = X_train.mean(dim=0, keepdim=True)
  s = X_train.std(dim=0, keepdim=True)
  X_train = (X_train - m) / s

  #################################################################################################

  X_validation_list = []
  y_validation_regression_list = []
  y_validation_classification_list = []
  y_validation_date = []
  for idx in range(row_cursor, row_cursor + validation_size):
    sequence_data = df.iloc[idx: idx + sequence_size].values  # sequence_data.shape: (sequence_size, 5)
    X_validation_list.append(torch.from_numpy(sequence_data))
    y_validation_regression_list.append(df.iloc[idx + sequence_size][target_column])
    y_validation_classification_list.append(
      1 if df.iloc[idx + sequence_size][target_column] >= df.iloc[idx + sequence_size - 1][target_column] else 0
    )
    y_validation_date.append(date_list[idx + sequence_size])
    row_cursor += 1

  X_validation = torch.stack(X_validation_list, dim=0).to(torch.float)
  y_validation_regression = torch.tensor(y_validation_regression_list, dtype=torch.float32) / y_normalizer
  y_validation_classification = torch.tensor(y_validation_classification_list, dtype=torch.int64)

  X_validation = (X_validation - m) / s
  #################################################################################################

  X_test_list = []
  y_test_regression_list = []
  y_test_classification_list = []
  y_test_date = []
  for idx in range(row_cursor, row_cursor + test_size):
    sequence_data = df.iloc[idx: idx + sequence_size].values  # sequence_data.shape: (sequence_size, 5)
    X_test_list.append(torch.from_numpy(sequence_data))
    y_test_regression_list.append(df.iloc[idx + sequence_size][target_column])
    y_test_classification_list.append(
      1 if df.iloc[idx + sequence_size][target_column] > df.iloc[idx + sequence_size - 1][target_column] else 0
    )
    y_test_date.append(date_list[idx + sequence_size])
    row_cursor += 1

  X_test = torch.stack(X_test_list, dim=0).to(torch.float)
  y_test_regression = torch.tensor(y_test_regression_list, dtype=torch.float32) / y_normalizer
  y_test_classification = torch.tensor(y_test_classification_list, dtype=torch.int64)

  X_test = (X_test - m) / s

  if is_regression:
    return (
      X_train, X_validation, X_test,
      y_train_regression, y_validation_regression, y_test_regression,
      y_train_date, y_validation_date, y_test_date
    )
  else:
    return (
      X_train, X_validation, X_test,
      y_train_classification, y_validation_classification, y_test_classification,
      y_train_date, y_validation_date, y_test_date
    )


if __name__ == "__main__":
  is_regression = False

  X_train, X_validation, X_test, y_train, y_validation, y_test, y_train_date, y_validation_date, y_test_date \
    = get_cryptocurrency_data(
    sequence_size=10, validation_size=100, test_size=10,
    target_column='Close', y_normalizer=1.0e7, is_regression=is_regression
  )

  train_crypto_currency_dataset = CryptoCurrencyDataset(X=X_train, y=y_train, is_regression=is_regression)
  validation_crypto_currency_dataset = CryptoCurrencyDataset(X=X_validation, y=y_validation, is_regression=is_regression)
  test_crypto_currency_dataset = CryptoCurrencyDataset(X=X_test, y=y_test, is_regression=is_regression)

  train_data_loader = DataLoader(
    dataset=train_crypto_currency_dataset,
    batch_size=32,
    shuffle=True,
    drop_last=True
  )

  for idx, batch in enumerate(train_data_loader):
    input, target = batch
    print("{0} - {1}: {2}, {3}".format(idx, input.shape, target.shape, target))



# P 고찰내용
#### dataset을 사용하지 않은 방식과의 차이점
1. 데이터 처리 방식의 차이
* 기존 버전은 데이터배치나 셔플링 등을 수작업으로 처리 하지만 dataset을 활용하면 자동으로 한다.
2. dataset을 활용하면 중복된 코드를 피할 수 있다.
3. 코드의 가독성이 좋아진다
# 숙제후기
숙제 1에서 텐서를 다루는 방식을 배우고 숙제 2에서는 배운 방식을 활용해서 데이터를 정제하고 분할하고 머신러닝을 위한 준비 과정을 배웠다고 느꼇습니다. 모든 내용을 제걸로 만들지는 못 했지만 최대한 이해하고 직접 코드를 수행해보며 정규화의 과정과 각 데이터별로 적합한 형태로 변환하는 과정 등을 배워 흥미로웠습니다.